# Introdução

Esse código, comentado e explicado em português, foi feito para pessoas que tem interesse em aprender mais sobre o uso de Redes Neurais Covolucionais, mas que ainda não tenham tanta facilidade com inglês.

Caso tenha alguma sugestão de melhoria coloque nos comentários.

MNIST é uma base de dados com números "desenhados" e com seus respectivos rotulos que indicam qual número esta desenhado. Nesse desafio, temos a base de dados com os rótulos que será usada para treinar o modelo(base de treino), e a sem rótulos, que devemos prever qual número será.

Para isso, no código foi usado, principalmente, a API keras(Tensorflow backend) e foi obtido uma acurácia de  99,47%

# Importações Iniciais

In [1]:
import numpy as np # Algebra linear
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline


import itertools
from sklearn.model_selection import train_test_split

from tensorflow.keras.utils import to_categorical #(one-hot-encoding)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, MaxPooling2D
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [2]:
# Importando a base de dados
train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [3]:
train.shape, test.shape # Modo como os dados estão estrutudados (42000 linhas, 785 colunas), (28000 linhas, 784 colunas) 

((42000, 785), (28000, 784))

In [4]:
train.head() 

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Como pode-se ver acima, cada coluna da linha representa um pixel da imagem, e o número no pixel representa a cor(RGB) dele.

OBS: As imagens dos números estão em escala de cinza. 

In [5]:
X_train = train.iloc[:,1:].values.astype('float32')
y_train = train['label'].values.astype('int32')
test = test.values.astype('float32')

del train

In [6]:
X_train = X_train / 255.0 # Normalizando os dados de treino
test = test / 255.0 # Normalizando os dados de teste

In [7]:
# Converte o dataset de treino para o formato (num_imagens, img_linhas, img_colunas)
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
test = test.reshape(test.shape[0], 28, 28, 1)

In [8]:
y_train

array([1, 0, 1, ..., 7, 6, 9], dtype=int32)

É comum que bases de dados como essa, tenham os resultados das classificações de imagens como mostrado no ultimo código. Entretanto, o algoritmo utilizado lida melhor com outro tipo de dado, por isso, vamos categoriza-lo de uma forma diferente. Nesse caso, usaremos o one-hot encoding:

Exemplo:

    O que era apenas o número 2 vira um array de zeros no qual o indice 2 do array é igual a 1
    2 -> [0,0,1,0,0,0,0,0,0,0]

In [9]:
y_train = to_categorical(y_train)
y_train

array([[0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [10]:
np.random.seed(2) #usado para que seja possivel reproduzir a aleatoriedade do experimento com os mesmos resultados

Agora, vamos separar o conjunto de dados entre a parte que sera usada para treinar o modelo e a parte de teste, que sera utilizada para descobrirmos o quão bom o modelo é, antes de submetelo na Competição.

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size = 0.0001, random_state = 2)

# Rede Neural Covolucional

Quando se trata de imagens, as CNN(Covolutional Neural Network) ganham destaque. Isso ocorre pois, se utiliza de forma correta, esse modelo tende a ter um dos melhores indices de acerto na classificação de imagens.

**Assim, no modelo abaixo:**

1. Começamos com camadas de Convolução que recebem a imagem e aplicam filtros nela.
2. Colocamos a camada de pooling, a qual reduz as dimenções dos dados .
3. A camada de Dropout, simplesmente exclui parte do processamento na imagem até o momento. Ela é necessária para minimizar a possibilidade de superajustamento aos dados de treino.


4. Depois todas as camadas anteriores são repetidas para uma melhor performance.


5. A camada de Flattening, transforma os dados(que até então eram matrizes) em vetores, para que as camadas Densas consigam processar os dados.
6. A primeira camada Densa possui 256 "neurônios", que processam o dado e manda para a ultima camada.
7. Essa ultima camada possui 10 "neûronios" pois é a quantidade de saidas(os números de 0 á 9) que precisamos.

 Keras é uma biblioteca desenvolvida pela google que possui ótimas funções a serem ultilizadas 
 em algoritimos de aprendizado de maquina 

In [12]:
model = Sequential()

model.add(Conv2D(filters = 64, kernel_size = (5,5), padding = 'Same', activation = 'relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 64, kernel_size = (5,5), padding = 'Same', activation = 'relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(filters = 128, kernel_size = (3,3), padding = 'Same', activation = 'relu'))
model.add(Conv2D(filters = 128, kernel_size = (3,3), padding = 'Same', activation = 'relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.2))


model.add(Flatten())
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation = 'softmax'))

In [13]:
# Colocamos como metrica de acerto: acurácia
# Função de perda(exibe o quanto o algoritimo esta errado): categorical_crossentropy
# optimizer: adam (um dos melhores optimizadores atualmente)
model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
epochs = 38 # Número epocas do modelo
batch_size = 64 

# Aumento de Dados
   Essa tecnica pode ser utilizada para aumentar a quantidade de dados de forma artificial. No codigo abaixo, por exemplo, essa tecnica foi empregada pegando os dados das imagens originais do dataset, e gerando novas imagens modificadas.

**Caracteristicas modificadas:**
* Zoom da imagem
* Inclinação/rotação da imagem(Nesse caso, temos que tomar cuidado com a rotação pois o número 6 pode ser confundido com o 9 ou o contrário)
* Centro da imagem
* Largura
* Altura

In [15]:
datagen = ImageDataGenerator(featurewise_center=False,
                            samplewise_center=False,
                            featurewise_std_normalization=False,
                            samplewise_std_normalization=False,
                            zca_whitening=False,
                            rotation_range=10,
                            zoom_range=0.1,
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            horizontal_flip=False,
                            vertical_flip=False)
datagen.fit(X_train)

In [16]:
# Ajusta e Treina o modelo
history = model.fit_generator(datagen.flow(X_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (X_test,y_test),
                              verbose = 2, steps_per_epoch=X_train.shape[0] // batch_size)

Epoch 1/38
656/656 - 18s - loss: 0.2255 - accuracy: 0.9289 - val_loss: 1.8357e-04 - val_accuracy: 1.0000
Epoch 2/38
656/656 - 16s - loss: 0.0783 - accuracy: 0.9756 - val_loss: 0.1881 - val_accuracy: 0.8000
Epoch 3/38
656/656 - 19s - loss: 0.0588 - accuracy: 0.9820 - val_loss: 0.0162 - val_accuracy: 1.0000
Epoch 4/38
656/656 - 16s - loss: 0.0539 - accuracy: 0.9837 - val_loss: 0.0624 - val_accuracy: 1.0000
Epoch 5/38
656/656 - 17s - loss: 0.0462 - accuracy: 0.9864 - val_loss: 0.3172 - val_accuracy: 0.8000
Epoch 6/38
656/656 - 17s - loss: 0.0426 - accuracy: 0.9870 - val_loss: 0.1666 - val_accuracy: 0.8000
Epoch 7/38
656/656 - 18s - loss: 0.0368 - accuracy: 0.9885 - val_loss: 0.0142 - val_accuracy: 1.0000
Epoch 8/38
656/656 - 17s - loss: 0.0373 - accuracy: 0.9886 - val_loss: 0.0550 - val_accuracy: 1.0000
Epoch 9/38
656/656 - 17s - loss: 0.0323 - accuracy: 0.9899 - val_loss: 0.0087 - val_accuracy: 1.0000
Epoch 10/38
656/656 - 18s - loss: 0.0307 - accuracy: 0.9908 - val_loss: 0.0675 - val_ac

In [17]:
# Usa o modelo treinado para prever e classificar os dados sem classificação
# nesse caso, os dados que serão submetidos na competição.
results = model.predict(test) 

# O algoritimo retorna uma lista de probabilidades pra cada número

In [18]:
# Seleciona os indices com maior probabilidade
results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")

In [19]:
# Gera um dataframe com o Id e a resposta de cada imagem
output = pd.concat([pd.Series(range(1,28001), name = 'ImageId'), results], axis = 1)
output

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,0
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9


In [20]:
#Para finalizar, vamos gerar um aquivo csv a partir do dataframe.
output.to_csv('submission.csv', index=False)

# Fontes:

* https://keras.io/
* https://www.kaggle.com/yassineghouzam/introduction-to-cnn-keras-0-997-top-6
* https://www.kaggle.com/elcaiseri/mnist-simple-cnn-keras-accuracy-0-99-top-1